#  MABe Challenge - Social Action Recognition in Mice

### Overview
> In this competition, you’ll develop machine learning models to recognize behaviors in mice based on their movements, providing new insights into animal social structures and advancing behavioral science research.

### Description
> Animal social behavior is complex. Species from ants to wolves to mice form social groups where they build nests, raise their young, care for their groupmates, and defend their territory. Studying these behaviors teaches us about the brain and the evolution of behavior, but the work has usually required subjective, time-consuming documentation of animals' actions. ML advancements now let us automate this process, supporting large-scale behavioral studies in the wild and in the lab.

> But even automated systems suffer from limited training data and poor generalizability. In current methods, an experimenter must hand-label hundreds of new training examples to automate recognition of a new behavior, which makes studying rare behaviors a challenge. And models trained within one research group usually fail when applied to data from other studies, meaning there is no guarantee that two labs are really studying the same behavior.

> This competition challenges you to build models to identify over 30 different social and non-social behaviors in pairs and groups of co-housed mice, based on markerless motion capture of their movements in top-down video recordings. The dataset includes over 400 hours of footage from 20+ behavioral recording systems, all carefully labeled frame-by-frame by experts. Your goal is to recognize these behaviors as accurately as a trained human observer while overcoming the inherent variability arising from the use of different data collection equipment and motion capture pipelines.

Your work will help scientists automate behavior analysis and better understand animal social structures. These models may be deployed across numerous labs, in neuroscience, computational biology, ethology, and ecology, to create a foundation for future ML and behavior research.

This notebook tackles the MABe challenge by building a separate GBDT ensemble model for each unique `body_parts_tracked` configuration. 

**Core Strategy:**
1.  **Grouped by Tracker:** Loop through each `body_parts_tracked` string.
2.  **FPS-Aware Features:** Generate advanced temporal and spatial features (`transform_single`, `transform_pair`). All window sizes, lags, and spans are *scaled* by the video's FPS to ensure features are time-invariant (`_scale` function).
3.  **Stratified Subsampling:** Use a `StratifiedSubsetClassifier` wrapper to train the GBDTs on a large, stratified subsample of the full dataset to manage memory and time.
4.  **Ensemble Model:** For each behavior, train an ensemble of LGBM, XGBoost, and CatBoost models.
5.  **Adaptive Prediction:** Use temporal smoothing, adaptive per-action probability thresholds, and minimum duration filtering to generate final event segments (`predict_multiclass_adaptive`).

## 1. IMPORTS & CONFIGURATION

In [ ]:
import pandas as pd
import numpy as np
import polars as pl
from tqdm import tqdm
import itertools
import warnings
import json
import os, random
import gc
from collections import defaultdict
from scipy import signal, stats

# --- Models ---
import lightgbm
from sklearn.base import ClassifierMixin, BaseEstimator, clone
from sklearn.pipeline import make_pipeline

# --- Model Selection & Metrics ---
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score

# --- Optional Imports ---
try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except:
    XGBOOST_AVAILABLE = False
    
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except:
    CATBOOST_AVAILABLE = False

# --- Configuration ---
validate_or_submit = 'submit'
verbose = True
SEED = 1234

# Body parts to drop from high-dimensional trackers (e.g., headpieces)
DROP_BODY_PARTS = [
    'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
    'spine_1', 'spine_2', 'tail_middle_1', 'tail_middle_2', 'tail_midpoint'
]

warnings.filterwarnings('ignore')

# --- Seeding ---
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

## 2. UTILITY: STRATIFIED SUBSET CLASSIFIER

In [ ]:
class StratifiedSubsetClassifier(ClassifierMixin, BaseEstimator):
    """A wrapper to train an estimator on a stratified subsample of data."""
    def __init__(self, estimator, n_samples=None):
        self.estimator = estimator
        self.n_samples = n_samples  # if None → no subsampling

    def _to_numpy(self, X):
        try:
            return X.to_numpy(np.float32, copy=False)
        except AttributeError:
            return np.asarray(X, dtype=np.float32)

    def fit(self, X, y):
        Xn = self._to_numpy(X)
        y = np.asarray(y).ravel()

        # Handle rare cases where labels might be [0, 2]
        uniq = np.unique(y[~pd.isna(y)])
        if set(uniq.tolist()) == {0, 2}:
            y = (y > 0).astype(np.int8)

        # If n_samples is None or data is small, fit on full data
        if self.n_samples is None or len(Xn) <= int(self.n_samples):
            self.estimator.fit(Xn, y)
        else:
            # Fit on a stratified subset
            sss = StratifiedShuffleSplit(n_splits=1, train_size=int(self.n_samples), random_state=42)
            try:
                idx, _ = next(sss.split(np.zeros_like(y), y))
                self.estimator.fit(Xn[idx], y[idx])
            except Exception:
                # Fallback for cases where stratification fails (e.g., too few samples of one class)
                step = max(len(Xn) // int(self.n_samples), 1)
                self.estimator.fit(Xn[::step], y[::step])

        try:
            self.classes_ = np.asarray(self.estimator.classes_)
        except Exception:
            self.classes_ = np.unique(y)
        return self

    def predict_proba(self, X):
        Xn = self._to_numpy(X)
        try:
            P = self.estimator.predict_proba(Xn)
        except Exception:
            # Handle models that failed training or have only one class
            if len(self.classes_) == 1:
                n = len(Xn)
                c = int(self.classes_[0])
                if c == 1:
                    return np.column_stack([np.zeros(n, dtype=np.float32), np.ones(n, dtype=np.float32)])
                else:
                    return np.column_stack([np.ones(n, dtype=np.float32), np.zeros(n, dtype=np.float32)])
            return np.full((len(Xn), 2), 0.5, dtype=np.float32)

        P = np.asarray(P)
        # Standardize output shape to (n_samples, 2)
        if P.ndim == 1:
            P1 = P.astype(np.float32)
            return np.column_stack([1.0 - P1, P1])
        if P.shape[1] == 1 and len(self.classes_) == 2:
            P1 = P[:, 0].astype(np.float32)
            return np.column_stack([1.0 - P1, P1])
        return P

    def predict(self, X):
        Xn = self._to_numpy(X)
        try:
            return self.estimator.predict(Xn)
        except Exception:
            return np.argmax(self.predict_proba(Xn), axis=1)

## 3. OFFICIAL SCORING FUNCTION (HIDDEN)

In [ ]:
class HostVisibleError(Exception):
    pass

def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set)
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set)

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()
        active_labels: set[str] = set(json.loads(active_labels))
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set)

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts():
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                continue
           
            new_frames = set(range(row['start_frame'], row['stop_frame']))
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int)
    fns = defaultdict(int)
    fps = defaultdict(int)
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)

def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:
    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)

## 4. DATA LOADING & PREPARATION

In [ ]:
BASE_PATH = '/kaggle/input/MABe-mouse-behavior-detection'

train = pd.read_csv(f'{BASE_PATH}/train.csv')
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)

test = pd.read_csv(f'{BASE_PATH}/test.csv')

# Get a unique list of all body part tracking configurations
body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

def generate_mouse_data(dataset, traintest, traintest_directory=None, generate_single=True, generate_pair=True):
    """
    Generator function to load and process video data one by one.
    Yields data for single mice (agent=target) and mouse pairs (agent!=target).
    """
    assert traintest in ['train', 'test']
    if traintest_directory is None:
        traintest_directory = f"{BASE_PATH}/{traintest}_tracking"
    
    for _, row in dataset.iterrows():
        lab_id = row.lab_id
        video_id = row.video_id

        if not isinstance(row.behaviors_labeled, str):
            if verbose: print(f'No labeled behaviors: {lab_id} {video_id}')
            continue

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        try:
            vid = pd.read_parquet(path)
        except FileNotFoundError:
            if verbose: print(f"File not found: {path}")
            continue

        # Filter out optional, high-dim body parts
        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@DROP_BODY_PARTS)")
        
        # Pivot to a (frame, mouse, bodypart, coord) structure
        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])
        del vid; gc.collect()

        if pvid.isna().any().any():
            if verbose and traintest == 'test': print(f'video with missing values: {video_id} ({len(pvid)} frames)')
        else:
            if verbose and traintest == 'test': print(f'video with all values: {video_id} ({len(pvid)} frames)')
        
        # Reorder levels to (mouse, bodypart, coord) and normalize by pixel density
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
        pvid /= row.pix_per_cm_approx  # Convert pixels to cm

        # Load behaviors for this video
        vid_behaviors = json.loads(row.behaviors_labeled)
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
        
        annot = None
        if traintest == 'train':
            try:
                annot_path = path.replace('train_tracking', 'train_annotation')
                annot = pd.read_parquet(annot_path)
            except FileNotFoundError:
                if verbose: print(f"Annotation not found, skipping: {annot_path}")
                continue

        # --- Process Single Mouse (self-behaviors) ---
        if generate_single:
            vid_behaviors_subset = vid_behaviors.query("target == 'self'")
            for mouse_id_str in np.unique(vid_behaviors_subset.agent):
                try:
                    mouse_id = int(mouse_id_str[-1])
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("agent == @mouse_id_str").action)
                    
                    # Select this mouse's pose data
                    single_mouse = pvid.loc[:, mouse_id]
                    assert len(single_mouse) == len(pvid)
                    
                    single_mouse_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': mouse_id_str,
                        'target_id': 'self',
                        'video_frame': single_mouse.index,
                        'frames_per_second': row.frames_per_second
                    })
                    
                    if traintest == 'train':
                        # Create frame-by-frame label matrix
                        single_mouse_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=single_mouse.index)
                        annot_subset = annot.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")
                        for _, annot_row in annot_subset.iterrows():
                            single_mouse_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'single', single_mouse, single_mouse_meta, single_mouse_label
                    else:
                        if verbose: print(f'- test single: {video_id} {mouse_id}')
                        yield 'single', single_mouse, single_mouse_meta, vid_agent_actions
                except KeyError: # Mouse ID not in pose data
                    pass

        # --- Process Mouse Pairs (social behaviors) ---
        if generate_pair:
            vid_behaviors_subset = vid_behaviors.query("target != 'self'")
            if len(vid_behaviors_subset) > 0:
                for agent, target in itertools.permutations(np.unique(pvid.columns.get_level_values('mouse_id')), 2):
                    agent_str = f"mouse{agent}"
                    target_str = f"mouse{target}"
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("(agent == @agent_str) & (target == @target_str)").action)
                    
                    # Select pose data for agent 'A' and target 'B'
                    mouse_pair = pd.concat([pvid[agent], pvid[target]], axis=1, keys=['A', 'B'])
                    assert len(mouse_pair) == len(pvid)
                    
                    mouse_pair_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': agent_str,
                        'target_id': target_str,
                        'video_frame': mouse_pair.index,
                        'frames_per_second': row.frames_per_second
                    })
                    
                    if traintest == 'train':
                        # Create frame-by-frame label matrix
                        mouse_pair_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=mouse_pair.index)
                        annot_subset = annot.query("(agent_id == @agent) & (target_id == @target)")
                        for _, annot_row in annot_subset.iterrows():
                            mouse_pair_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'pair', mouse_pair, mouse_pair_meta, mouse_pair_label
                    else:
                        if verbose: print(f'- test pair: {video_id} {agent} -> {target}')
                        yield 'pair', mouse_pair, mouse_pair_meta, vid_agent_actions

## 5. FPS-AWARE FEATURE ENGINEERING

In [ ]:
# --- FPS Scaling Utilities ---
# These functions are CRITICAL. They convert a window/lag size defined 
# at a reference (30 FPS) to the correct size for the current video's FPS.
# This makes features robust to variable frame rates.

def _scale(n_frames_at_30fps, fps, ref=30.0):
    """Scale a frame count defined at 30 fps to the current video's fps."""
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    """Signed version of _scale for forward/backward shifts."""
    if n_frames_at_30fps == 0:
        return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

def _fps_from_meta(meta_df, fallback_lookup, default_fps=30.0):
    """Safely get the FPS for the current video."""
    if 'frames_per_second' in meta_df.columns and pd.notnull(meta_df['frames_per_second']).any():
        return float(meta_df['frames_per_second'].iloc[0])
    # Fallback for test set where FPS might be missing in sample
    vid = meta_df['video_id'].iloc[0]
    return float(fallback_lookup.get(vid, default_fps))

# --- Advanced Feature Sub-routines (FPS-Aware) ---

def add_curvature_features(X, center_x, center_y, fps):
    """Trajectory curvature (window lengths scaled by fps)."""
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()

    cross_prod = vel_x * acc_y - vel_y * acc_x
    vel_mag = np.sqrt(vel_x**2 + vel_y**2)
    curvature = np.abs(cross_prod) / (vel_mag**3 + 1e-6)

    for w in [30, 60]:
        ws = _scale(w, fps)
        X[f'curv_mean_{w}'] = curvature.rolling(ws, min_periods=max(1, ws // 6)).mean()

    angle = np.arctan2(vel_y, vel_x)
    angle_change = np.abs(angle.diff())
    w = 30
    ws = _scale(w, fps)
    X[f'turn_rate_{w}'] = angle_change.rolling(ws, min_periods=max(1, ws // 6)).sum()
    return X

def add_multiscale_features(X, center_x, center_y, fps):
    """Multi-scale temporal features (speed in cm/s; windows scaled by fps)."""
    # displacement (cm) * fps = speed (cm/s)
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)

    scales = [10, 40, 160] # ~0.3s, ~1.3s, ~5.3s at 30fps
    for scale in scales:
        ws = _scale(scale, fps)
        if len(speed) >= ws:
            X[f'sp_m{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).mean()
            X[f'sp_s{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).std()

    if len(scales) >= 2 and f'sp_m{scales[0]}' in X.columns and f'sp_m{scales[-1]}' in X.columns:
        X['sp_ratio'] = X[f'sp_m{scales[0]}'] / (X[f'sp_m{scales[-1]}'] + 1e-6)
    return X

def add_state_features(X, center_x, center_y, fps):
    """Behavioral state transitions; bins adjusted so semantics are fps-invariant."""
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)  # cm/s
    w_ma = _scale(15, fps) # 0.5s moving average
    speed_ma = speed.rolling(w_ma, min_periods=max(1, w_ma // 3)).mean()

    try:
        # Original bins (cm/frame) at 30fps: [-inf, 0.5, 2.0, 5.0, inf]
        # Convert to cm/s by multiplying by fps.
        bins = [-np.inf, 0.5 * fps, 2.0 * fps, 5.0 * fps, np.inf]
        speed_states = pd.cut(speed_ma, bins=bins, labels=[0, 1, 2, 3]).astype(float)

        for window in [60, 120]: # 2s, 4s
            ws = _scale(window, fps)
            if len(speed_states) >= ws:
                for state in [0, 1, 2, 3]:
                    X[f's{state}_{window}'] = ((speed_states == state).astype(float)
                                                  .rolling(ws, min_periods=max(1, ws // 6)).mean())
                state_changes = (speed_states != speed_states.shift(1)).astype(float)
                X[f'trans_{window}'] = state_changes.rolling(ws, min_periods=max(1, ws // 6)).sum()
    except Exception: # pd.cut can fail if all values are identical
        pass
    return X

def add_longrange_features(X, center_x, center_y, fps):
    """Long-range temporal features (windows & spans scaled by fps)."""
    for window in [120, 240]: # 4s, 8s
        ws = _scale(window, fps)
        if len(center_x) >= ws:
            X[f'x_ml{window}'] = center_x.rolling(ws, min_periods=max(5, ws // 6)).mean()
            X[f'y_ml{window}'] = center_y.rolling(ws, min_periods=max(5, ws // 6)).mean()

    for span in [60, 120]: # 2s, 4s EWM spans
        s = _scale(span, fps)
        X[f'x_e{span}'] = center_x.ewm(span=s, min_periods=1).mean()
        X[f'y_e{span}'] = center_y.ewm(span=s, min_periods=1).mean()

    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)  # cm/s
    for window in [60, 120]:
        ws = _scale(window, fps)
        if len(speed) >= ws:
            X[f'sp_pct{window}'] = speed.rolling(ws, min_periods=max(5, ws // 6)).rank(pct=True)
    return X

def add_interaction_features(X, mouse_pair, avail_A, avail_B, fps):
    """Social interaction features (windows scaled by fps)."""
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X

    rel_x = mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x']
    rel_y = mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y']
    rel_dist = np.sqrt(rel_x**2 + rel_y**2)

    A_vx = mouse_pair['A']['body_center']['x'].diff()
    A_vy = mouse_pair['A']['body_center']['y'].diff()
    B_vx = mouse_pair['B']['body_center']['x'].diff()
    B_vy = mouse_pair['B']['body_center']['y'].diff()

    # Cosine similarity between agent's velocity and vector to target
    A_lead = (A_vx * rel_x + A_vy * rel_y) / (np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    # Cosine similarity between target's velocity and vector to agent
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (np.sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)

    for window in [30, 60]: # 1s, 2s
        ws = _scale(window, fps)
        X[f'A_ld{window}'] = A_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()
        X[f'B_ld{window}'] = B_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()

    approach = -rel_dist.diff()  # positive = decreasing distance
    chase = approach * B_lead # Agent approaches & Target leads (runs away from agent)
    w = 30 # 1s
    ws = _scale(w, fps)
    X[f'chase_{w}'] = chase.rolling(ws, min_periods=max(1, ws // 6)).mean()

    for window in [60, 120]: # 2s, 4s
        ws = _scale(window, fps)
        A_sp = np.sqrt(A_vx**2 + A_vy**2)
        B_sp = np.sqrt(B_vx**2 + B_vy**2)
        X[f'sp_cor{window}'] = A_sp.rolling(ws, min_periods=max(1, ws // 6)).corr(B_sp)
    return X


In [ ]:
# --- Main Feature Transformation Functions ---

def transform_single(single_mouse, body_parts_tracked, fps):
    """Master feature engineering pipeline for single mouse (self-behaviors)."""
    available_body_parts = single_mouse.columns.get_level_values(0)

    # Base distance features (squared distances, cm^2)
    X = pd.DataFrame({
        f"{p1}+{p2}": np.square(single_mouse[p1] - single_mouse[p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.combinations(body_parts_tracked, 2)
        if p1 in available_body_parts and p2 in available_body_parts
    })
    X = X.reindex(columns=[f"{p1}+{p2}" for p1, p2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    # Speed-like features (lagged displacement, cm^2)
    if all(p in single_mouse.columns for p in ['ear_left', 'ear_right', 'tail_base']):
        lag = _scale(10, fps) # 0.33s lag
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag)
        speeds = pd.DataFrame({
            'sp_lf': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
            'sp_rt': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
            'sp_lf2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
            'sp_rt2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    # Body angle (orientation)
    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        X['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (
            np.sqrt(v1['x']**2 + v1['y']**2) * np.sqrt(v2['x']**2 + v2['y']**2) + 1e-6)

    # Core temporal features
    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']

        for w in [5, 15, 30, 60]: # ~0.16s, 0.5s, 1s, 2s
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'cx_m{w}'] = cx.rolling(ws, **roll).mean()
            X[f'cy_m{w}'] = cy.rolling(ws, **roll).mean()
            X[f'cx_s{w}'] = cx.rolling(ws, **roll).std()
            X[f'cy_s{w}'] = cy.rolling(ws, **roll).std()
            X[f'x_rng{w}'] = cx.rolling(ws, **roll).max() - cx.rolling(ws, **roll).min()
            X[f'y_rng{w}'] = cy.rolling(ws, **roll).max() - cy.rolling(ws, **roll).min()
            X[f'disp{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).sum()**2 +
                                     cy.diff().rolling(ws, min_periods=1).sum()**2)
            X[f'act{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).var() +
                                   cy.diff().rolling(ws, min_periods=1).var())

        # Advanced features (all FPS-scaled internally)
        X = add_curvature_features(X, cx, cy, fps)
        X = add_multiscale_features(X, cx, cy, fps)
        X = add_state_features(X, cx, cy, fps)
        X = add_longrange_features(X, cx, cy, fps)

    # Nose-tail features (cm)
    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                          (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        for lag in [10, 20, 40]: # ~0.33s, 0.66s, 1.33s
            l = _scale(lag, fps)
            X[f'nt_lg{lag}'] = nt_dist.shift(l)
            X[f'nt_df{lag}'] = nt_dist - nt_dist.shift(l)

    # Ear features (cm)
    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                        (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        for off in [-20, -10, 10, 20]: # ~ +/- 0.33s, 0.66s
            o = _scale_signed(off, fps)
            X[f'ear_o{off}'] = ear_d.shift(-o)  
        w = _scale(30, fps) # 1s window
        X['ear_con'] = ear_d.rolling(w, min_periods=1, center=True).std() / \
                       (ear_d.rolling(w, min_periods=1, center=True).mean() + 1e-6)

    return X.astype(np.float32, copy=False)

def transform_pair(mouse_pair, body_parts_tracked, fps):
    """Master feature engineering pipeline for mouse pairs (social behaviors)."""
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    # Inter-mouse distances (squared distances, cm^2)
    X = pd.DataFrame({
        f"12+{p1}+{p2}": np.square(mouse_pair['A'][p1] - mouse_pair['B'][p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.product(body_parts_tracked, repeat=2)
        if p1 in avail_A and p2 in avail_B
    })
    X = X.reindex(columns=[f"12+{p1}+{p2}" for p1, p2 in itertools.product(body_parts_tracked, repeat=2)], copy=False)

    # Speed-like features (lagged displacement, cm^2)
    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps) # 0.33s lag
        shA = mouse_pair['A']['ear_left'].shift(lag)
        shB = mouse_pair['B']['ear_left'].shift(lag)
        speeds = pd.DataFrame({
            'sp_A': np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False),
            'sp_AB': np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False),
            'sp_B': np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    # Elongation (placeholder, as it's a single-mouse feature)
    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    # Relative orientation
    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        X['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            np.sqrt(dir_A['x']**2 + dir_A['y']**2) * np.sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)

    # Approach rate (cm^2 / 0.33s)
    if all(p in avail_A for p in ['nose']) and all(p in avail_B for p in ['nose']):
        cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        lag = _scale(10, fps)
        shA_n = mouse_pair['A']['nose'].shift(lag)
        shB_n = mouse_pair['B']['nose'].shift(lag)
        past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
        X['appr'] = cur - past # negative = approach

    # Distance bins (cm)
    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                     (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
        X['v_cls'] = (cd < 5.0).astype(float)
        X['cls']   = ((cd >= 5.0) & (cd < 15.0)).astype(float)
        X['med']   = ((cd >= 15.0) & (cd < 30.0)).astype(float)
        X['far']   = (cd >= 30.0).astype(float)

    # Temporal interaction features
    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd_full = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)

        for w in [5, 15, 30, 60]: # ~0.16s, 0.5s, 1s, 2s
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'd_m{w}']  = cd_full.rolling(ws, **roll).mean()
            X[f'd_s{w}']  = cd_full.rolling(ws, **roll).std()
            X[f'd_mn{w}'] = cd_full.rolling(ws, **roll).min()
            X[f'd_mx{w}'] = cd_full.rolling(ws, **roll).max()

            d_var = cd_full.rolling(ws, **roll).var()
            X[f'int{w}'] = 1 / (1 + d_var) # Interaction metric (low variance in distance)

            Axd = mouse_pair['A']['body_center']['x'].diff()
            Ayd = mouse_pair['A']['body_center']['y'].diff()
            Bxd = mouse_pair['B']['body_center']['x'].diff()
            Byd = mouse_pair['B']['body_center']['y'].diff()
            coord = Axd * Bxd + Ayd * Byd # Velocity dot product
            X[f'co_m{w}'] = coord.rolling(ws, **roll).mean()
            X[f'co_s{w}'] = coord.rolling(ws, **roll).std()

    # Nose-nose dynamics (cm)
    if 'nose' in avail_A and 'nose' in avail_B:
        nn_dist = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                          (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)
        for lag in [10, 20, 40]: # ~0.33s, 0.66s, 1.33s
            l = _scale(lag, fps)
            X[f'nn_lg{lag}']  = nn_dist.shift(l)
            X[f'nn_ch{lag}']  = nn_dist - nn_dist.shift(l)
            is_cl = (nn_dist < 10.0).astype(float)
            X[f'cl_ps{lag}']  = is_cl.rolling(l, min_periods=1).mean()

    # Velocity alignment
    if 'body_center' in avail_A and 'body_center' in avail_B:
        Avx = mouse_pair['A']['body_center']['x'].diff()
        Avy = mouse_pair['A']['body_center']['y'].diff()
        Bvx = mouse_pair['B']['body_center']['x'].diff()
        Bvy = mouse_pair['B']['body_center']['y'].diff()
        val = (Avx * Bvx + Avy * Bvy) / (np.sqrt(Avx**2 + Avy**2) * np.sqrt(Bvx**2 + Bvy**2) + 1e-6)

        for off in [-20, -10, 0, 10, 20]: # ~ +/- 0.33s, 0.66s
            o = _scale_signed(off, fps)
            X[f'va_{off}'] = val.shift(-o)

        w = _scale(30, fps) # 1s window
        X['int_con'] = cd_full.rolling(w, min_periods=1, center=True).std() / \
                       (cd_full.rolling(w, min_periods=1, center=True).mean() + 1e-6)

        # Advanced interaction (all FPS-scaled internally)
        X = add_interaction_features(X, mouse_pair, avail_A, avail_B, fps)

    return X.astype(np.float32, copy=False)

## 6. POST-PROCESSING & SUBMISSION

In [ ]:
# Default threshold for actions, can be tuned per-action
action_thresholds = defaultdict(lambda: 0.27)
# e.g., action_thresholds['attack'] = 0.4
# e.g., action_thresholds['sniff'] = 0.2

def predict_multiclass_adaptive(pred, meta, action_thresholds):
    """Converts frame-wise probabilities into event segments."""
    if pred.empty:
        return pd.DataFrame(columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])

    # 1. Temporal smoothing
    pred_smoothed = pred.rolling(window=5, min_periods=1, center=True).mean()
    
    # 2. Get most likely action per frame
    ama = np.argmax(pred_smoothed.values, axis=1)
    max_probs = pred_smoothed.max(axis=1).values
    
    # 3. Apply adaptive per-action thresholds
    threshold_mask = np.zeros(len(pred_smoothed), dtype=bool)
    for i, action in enumerate(pred_smoothed.columns):
        action_mask = (ama == i)
        threshold = action_thresholds.get(action, 0.27)
        threshold_mask |= (action_mask & (max_probs >= threshold))
    
    # 4. Set frames below threshold to -1 (no action)
    ama = np.where(threshold_mask, ama, -1)
    ama = pd.Series(ama, index=meta.video_frame)
    
    # 5. Find change-points (start/end of events)
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    
    # 6. Build submission dataframe
    mask = ama_changes.values >= 0
    if mask.any():
        mask[-1] = False # Last event is incomplete
    else:
        # No events found
        return pd.DataFrame(columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])

    start_indices = np.where(mask)[0]
    if len(start_indices) == 0:
        return pd.DataFrame(columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        
    stop_indices = start_indices + 1
    
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'].iloc[start_indices].values,
        'agent_id': meta_changes['agent_id'].iloc[start_indices].values,
        'target_id': meta_changes['target_id'].iloc[start_indices].values,
        'action': pred.columns[ama_changes.iloc[start_indices].values],
        'start_frame': ama_changes.index[start_indices],
        'stop_frame': ama_changes.index[stop_indices]
    })
    
    # --- Fix stop_frames that cross video boundaries (should be rare) ---
    stop_video_id = meta_changes['video_id'].iloc[stop_indices].values
    stop_agent_id = meta_changes['agent_id'].iloc[stop_indices].values
    stop_target_id = meta_changes['target_id'].iloc[stop_indices].values
    
    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]
        
        is_last_event = (i == len(submission_part) - 1)
        is_boundary = False
        if i < len(stop_video_id):
            is_boundary = (stop_video_id[i] != video_id or 
                           stop_agent_id[i] != agent_id or 
                           stop_target_id[i] != target_id)
        else:
            is_last_event = True # Handle if stop_indices was shorter

        if is_last_event or is_boundary:
            # Query for the max frame of this specific agent/target/video
            q_str = "(video_id == @video_id) & (agent_id == @agent_id) & (target_id == @target_id)"
            new_stop_frame = meta.query(q_str).video_frame.max() + 1
            submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
    
    # 7. Filter out very short events (likely noise)
    duration = submission_part.stop_frame - submission_part.start_frame
    submission_part = submission_part[duration >= 3].reset_index(drop=True)
    
    if len(submission_part) > 0:
        assert (submission_part.stop_frame > submission_part.start_frame).all(), 'stop <= start'
    
    if verbose: print(f'  actions found: {len(submission_part)}')
    return submission_part

## 7. MODEL DEFINITIONS

In [ ]:
def get_model_ensemble():
    """Defines the list of models to be trained."""
    models = []

    # --- Model 1: LGBM (Balanced) ---
    models.append(make_pipeline(
        StratifiedSubsetClassifier(
            lightgbm.LGBMClassifier(
                n_estimators=225, learning_rate=0.07, min_child_samples=40,
                num_leaves=31, subsample=0.8, colsample_bytree=0.8, verbose=-1,
                random_state=SEED, bagging_seed=SEED, feature_fraction_seed=SEED, data_random_seed=SEED
            ), 500_000, # Subsample to 500k rows
        )
    ))
    
    # --- Model 2: LGBM (Deeper) ---
    models.append(make_pipeline(
        StratifiedSubsetClassifier(
            lightgbm.LGBMClassifier(
                n_estimators=150, learning_rate=0.1, min_child_samples=20,
                num_leaves=63, max_depth=8, subsample=0.7, colsample_bytree=0.9,
                reg_alpha=0.1, reg_lambda=0.1, verbose=-1,
                random_state=SEED, bagging_seed=SEED, feature_fraction_seed=SEED, data_random_seed=SEED
            ), 450_000, # Subsample to 450k rows
        )
    ))

    # --- Model 3: LGBM (Larger Leaves) ---
    models.append(make_pipeline(
        StratifiedSubsetClassifier(
            lightgbm.LGBMClassifier(
                n_estimators=100, learning_rate=0.05, min_child_samples=30,
                num_leaves=127, max_depth=10, subsample=0.75, verbose=-1,
                random_state=SEED, bagging_seed=SEED, feature_fraction_seed=SEED, data_random_seed=SEED
            ), 400_000, # Subsample to 400k rows
        )
    ))
    
    # --- Model 4: XGBoost (if available) ---
    if XGBOOST_AVAILABLE:
        models.append(make_pipeline(
            StratifiedSubsetClassifier(
                XGBClassifier(
                    n_estimators=180, learning_rate=0.08, max_depth=6,
                    min_child_weight=5, subsample=0.8, colsample_bytree=0.8,
                    tree_method='hist', verbosity=0,
                    random_state=SEED
                ), 500_000,
            )
        ))
        
    # --- Model 5: CatBoost (if available) ---
    if CATBOOST_AVAILABLE:
        models.append(make_pipeline(
            StratifiedSubsetClassifier(
                CatBoostClassifier(
                    iterations=120, learning_rate=0.1, depth=6,
                    verbose=False, allow_writing_files=False,
                    random_seed=SEED
                ), 500_000,
            )
        ))
        
    return models

## 8. TRAINING & INFERENCE PIPELINE

In [ ]:
def train_and_predict_ensemble(body_parts_tracked_str, switch_tr, X_tr, label, meta):
    """
    Trains a full ensemble for one (tracker, type) combo and predicts on the test set.
    - `body_parts_tracked_str`: The JSON string for the tracker config.
    - `switch_tr`: 'single' or 'pair'.
    - `X_tr`, `label`: Training features and labels.
    - `meta`: Training metadata.
    """
    
    models = get_model_ensemble()
    
    # Convert to numpy and free memory
    X_tr_np = X_tr.to_numpy(np.float32, copy=False)
    del X_tr; gc.collect()

    # --- Train one ensemble for each action ---
    model_list = [] # List to store (action, [trained_model_1, ...])
    for action in label.columns:
        y_raw = label[action].to_numpy()
        mask = ~pd.isna(y_raw)
        y_action = y_raw[mask].astype(int)
        
        # Only train if we have positive examples
        if not (y_action == 0).all() and np.sum(y_action) >= 5:
            trained = []
            idx = np.flatnonzero(mask)
            for m in models:
                m_clone = clone(m)
                m_clone.fit(X_tr_np[idx], y_action)
                trained.append(m_clone)
            model_list.append((action, trained))

    del X_tr_np; gc.collect()

    # --- Prepare for Inference ---
    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in DROP_BODY_PARTS]

    # Get test videos matching this tracker config
    test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
    if test_subset.empty:
        if verbose: print("  No test videos for this tracker.")
        return
    
    # Create a lookup for FPS in case test metadata is missing it
    fps_lookup = (test_subset[['video_id', 'frames_per_second']]
                  .drop_duplicates('video_id')
                  .set_index('video_id')['frames_per_second']
                  .to_dict())

    generator = generate_mouse_data(
        test_subset, 'test',
        generate_single=(switch_tr == 'single'),
        generate_pair=(switch_tr == 'pair')
    )

    if verbose:
        print(f"  Inferring on {len(test_subset)} test video(s) with {len(models)} models...")

    # --- Inference Loop ---
    for switch_te, data_te, meta_te, actions_te in generator:
        assert switch_te == switch_tr
        try:
            fps_i = _fps_from_meta(meta_te, fps_lookup, default_fps=30.0)

            # Generate features for this test video
            if switch_te == 'single':
                X_te = transform_single(data_te, body_parts_tracked, fps_i).astype(np.float32)
            else:
                X_te = transform_pair(data_te, body_parts_tracked, fps_i).astype(np.float32)

            X_te_np = X_te.to_numpy(np.float32, copy=False)
            del X_te, data_te; gc.collect()

            # Predict probabilities for all relevant actions
            pred = pd.DataFrame(index=meta_te.video_frame)
            for action, trained in model_list:
                if action in actions_te:
                    # Average probabilities across the ensemble
                    probs = [m.predict_proba(X_te_np)[:, 1] for m in trained]
                    pred[action] = np.mean(probs, axis=0)

            del X_te_np; gc.collect()

            # Convert probabilities to submission segments
            if pred.shape[1] > 0:
                sub_part = predict_multiclass_adaptive(pred, meta_te, action_thresholds)
                submission_list.append(sub_part)
            elif verbose:
                print("  No models trained for required test actions.")

        except Exception as e:
            if verbose:
                print(f"  ERROR during inference: {str(e)[:100]}")
            try: del data_te 
            except Exception: pass
            gc.collect()

def robustify_submission(submission, dataset, traintest, traintest_directory=None):
    """Ensures all videos have at least one prediction and cleans overlaps."""
    if traintest_directory is None:
        traintest_directory = f"{BASE_PATH}/{traintest}_tracking"

    # 1. Filter invalid segments
    submission = submission[submission.start_frame < submission.stop_frame]

    # 2. Remove overlapping predictions for the same (video, agent, target)
    group_list = []
    for _, group in submission.groupby(['video_id', 'agent_id', 'target_id']):
        group = group.sort_values('start_frame')
        mask = np.ones(len(group), dtype=bool)
        last_stop = -1
        for i, (_, row) in enumerate(group.iterrows()):
            if row['start_frame'] < last_stop:
                mask[i] = False # This event overlaps with the previous one
            else:
                last_stop = row['stop_frame']
        group_list.append(group[mask])
    submission = pd.concat(group_list, ignore_index=True) if group_list else pd.DataFrame(columns=submission.columns)

    # 3. Add placeholder predictions for any videos with no predictions
    s_list = []
    predicted_videos = set(submission.video_id.unique())
    
    for _, row in dataset.iterrows():
        video_id = row['video_id']
        if video_id in predicted_videos:
            continue

        if verbose:
            print(f"Video {video_id} has no predictions, adding placeholder.")
        
        try:
            path = f"{traintest_directory}/{row['lab_id']}/{video_id}.parquet"
            vid = pd.read_parquet(path)
            vid_behaviors = eval(row['behaviors_labeled'])
            vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
            vid_behaviors = [b.split(',') for b in vid_behaviors]
            vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])

            start_frame = vid.video_frame.min()
            stop_frame = vid.video_frame.max() + 1

            # Add one tiny placeholder for the first labeled action
            if not vid_behaviors.empty:
                vb_row = vid_behaviors.iloc[0]
                s_list.append((video_id, vb_row['agent'], vb_row['target'], vb_row['action'], start_frame, start_frame + 1))
        except Exception as e:
             if verbose: print(f"Could not create placeholder for {video_id}: {e}")

    if len(s_list) > 0:
        placeholder_df = pd.DataFrame(s_list, columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        submission = pd.concat([submission, placeholder_df], ignore_index=True)

    submission = submission.reset_index(drop=True)
    return submission


## 9. MAIN EXECUTION LOOP

In [ ]:
submission_list = []
print(f"XGBoost: {XGBOOST_AVAILABLE}, CatBoost: {CATBOOST_AVAILABLE}\n")

# We skip list[0] as it's often '[]' (no parts tracked)
for section, body_parts_tracked_str in enumerate(body_parts_tracked_list[1:], 1):
    try:
        body_parts_tracked_names = json.loads(body_parts_tracked_str)
        print(f"--- Processing Tracker Config {section}/{len(body_parts_tracked_list)-1} ({len(body_parts_tracked_names)} body parts) ---")
        
        # Filter to only the body parts we use
        if len(body_parts_tracked_names) > 5:
            body_parts_tracked = [b for b in body_parts_tracked_names if b not in DROP_BODY_PARTS]
        else:
            body_parts_tracked = body_parts_tracked_names

        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]
        if train_subset.empty:
            print("  No training videos for this tracker. Skipping.")
            print()
            continue

        # Create FPS lookup for this training subset
        _fps_lookup = (train_subset[['video_id', 'frames_per_second']]
                       .drop_duplicates('video_id')
                       .set_index('video_id')['frames_per_second']
                       .to_dict())

        # --- Load all data for this tracker config ---
        single_list, single_label_list, single_meta_list = [], [], []
        pair_list, pair_label_list, pair_meta_list = [], [], []

        data_generator = generate_mouse_data(train_subset, 'train')
        for switch, data, meta, label in data_generator:
            if switch == 'single':
                single_list.append(data)
                single_meta_list.append(meta)
                single_label_list.append(label)
            else:
                pair_list.append(data)
                pair_meta_list.append(meta)
                pair_label_list.append(label)

        # --- Process 'single' mouse data (self-behaviors) ---
        if len(single_list) > 0:
            single_feats_parts = []
            for data_i, meta_i in zip(single_list, single_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup, default_fps=30.0)
                Xi = transform_single(data_i, body_parts_tracked, fps_i).astype(np.float32)
                single_feats_parts.append(Xi)

            X_tr = pd.concat(single_feats_parts, axis=0, ignore_index=True)
            single_label = pd.concat(single_label_list, axis=0, ignore_index=True)
            single_meta  = pd.concat(single_meta_list,  axis=0, ignore_index=True)

            del single_list, single_label_list, single_meta_list, single_feats_parts
            gc.collect()

            print(f"  Single: Loaded {X_tr.shape[0]} frames, {X_tr.shape[1]} features")
            train_and_predict_ensemble(body_parts_tracked_str, 'single', X_tr, single_label, single_meta)

            del X_tr, single_label, single_meta
            gc.collect()
        else:
             print("  Single: No data.")

        # --- Process 'pair' mouse data (social behaviors) ---
        if len(pair_list) > 0:
            pair_feats_parts = []
            for data_i, meta_i in zip(pair_list, pair_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup, default_fps=30.0)
                Xi = transform_pair(data_i, body_parts_tracked, fps_i).astype(np.float32)
                pair_feats_parts.append(Xi)

            X_tr = pd.concat(pair_feats_parts, axis=0, ignore_index=True)
            pair_label = pd.concat(pair_label_list, axis=0, ignore_index=True)
            pair_meta  = pd.concat(pair_meta_list,  axis=0, ignore_index=True)

            del pair_list, pair_label_list, pair_meta_list, pair_feats_parts
            gc.collect()

            print(f"  Pair: Loaded {X_tr.shape[0]} frames, {X_tr.shape[1]} features")
            train_and_predict_ensemble(body_parts_tracked_str, 'pair', X_tr, pair_label, pair_meta)

            del X_tr, pair_label, pair_meta
            gc.collect()
        else:
            print("  Pair: No data.")

    except Exception as e:
        print(f'*** CRITICAL ERROR in main loop: {str(e)[:100]} ***')

    gc.collect()
    print()

## 10. FINALIZE SUBMISSION

In [ ]:
if len(submission_list) > 0:
    submission = pd.concat(submission_list, ignore_index=True)
else:
    # Create a dummy submission if all models failed
    print("No predictions generated, creating dummy submission.")
    submission = pd.DataFrame({
        'video_id': [438887472],
        'agent_id': ['mouse1'],
        'target_id': ['self'],
        'action': ['rear'],
        'start_frame': [278],
        'stop_frame': [500]
    })

submission_robust = robustify_submission(submission, test, 'test')
submission_robust.index.name = 'row_id'
submission_robust.to_csv('submission.csv')

print(f"\nSubmission created: {len(submission_robust)} total predictions")
print("Done.")

### Connect with Me  

Feel free to follow me on these platforms:  

[![GitHub](https://img.shields.io/badge/GitHub-181717?style=for-the-badge&logo=github&logoColor=white)](https://github.com/AdilShamim8)  
[![LinkedIn](https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/adilshamim8)  
[![Twitter](https://img.shields.io/badge/Twitter-1DA1F2?style=for-the-badge&logo=twitter&logoColor=white)](https://x.com/adil_shamim8)  